In [ ]:
# import packages
# import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn.model_selection import train_test_split, cross_validate
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures, StandardScaler, LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, roc_curve, auc, precision_recall_curve, make_scorer, recall_score, brier_score_loss
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline

# libraries for cleaning
import re
import nltk
nltk.download("stopwords") # helps us get rid of stop words
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

# tensor flow
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv("/content/drive/MyDrive/UBDC_2021/Final/combined_df_label2.csv")
df = df.iloc[:,1:]

Mounted at /content/drive


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
df.head()

,coordinates,created_at,hashtags,media,urls,favorite_count,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,lang,place,possibly_sensitive,retweet_count,retweet_id,retweet_screen_name,source,text,tweet_url,user_created_at,user_screen_name,user_default_profile_image,user_description,user_favourites_count,user_followers_count,user_friends_count,user_listed_count,user_location,user_name,user_screen_name.1,user_statuses_count,user_time_zone,user_urls,user_verified,label
0,NaN,Fri May 01 03:24:58 +0000 2020,NaN,https://twitter.com/worldnews911/status/125606...,https://news.familysouq.net/coronavirus-outbre...,0,1256062026260123648,NaN,NaN,NaN,en,NaN,False,0,NaN,NaN,"<a href=""http://publicize.wp.com/"" rel=""nofoll...",Coronavirus Outbreak LIVE Updates: Odisha conf...,https://twitter.com/MedicalWL/status/125606202...,Fri Feb 15 21:30:16 +0000 2019,MedicalWL,False,The Medical Weight Loss,0,104,0,1,NaN,The Medical Weight Loss,MedicalWL,62227,NaN,https://themedicalweightloss.com/,False,0
1,NaN,Fri May 01 11:35:09 +0000 2020,NaN,https://twitter.com/nytimes/status/12561853840...,https://nyti.ms/2SG4zSv https://nyti.ms/2YlMmN...,89,1256185384050065408,NaN,NaN,NaN,en,NaN,False,40,NaN,NaN,"<a href=""https://mobile.twitter.com"" rel=""nofo...","For today, May 1, here’s how to follow the lat...",https://twitter.com/nytimes/status/12561853840...,Fri Mar 02 20:41:42 +0000 2007,nytimes,False,News tips? Share them here: https://t.co/ghL9O...,18518,49905205,820,209461,New York City,The New York Times,nytimes,435062,NaN,http://www.nytimes.com/,True,0
2,NaN,Fri May 01 04:51:25 +0000 2020,NaN,https://twitter.com/biswaranjan412/status/1256...,https://ajkatajanews.com/coronavirus-mumbai-de...,0,1256083784686096385,NaN,NaN,NaN,in,NaN,False,0,NaN,NaN,"<a href=""http://publicize.wp.com/"" rel=""nofoll...",Coronavirus | Mumbai Delhi Coronavirus News | ...,https://twitter.com/biswaranjan412/status/1256...,Wed Oct 21 18:53:25 +0000 2015,biswaranjan412,False,NaN,2,33,64,0,NaN,biswaranjan mohapatr,biswaranjan412,20082,NaN,NaN,False,0
3,NaN,Thu Apr 30 10:07:14 +0000 2020,NaN,https://twitter.com/Ankit68419889/status/12558...,https://jankarinews.in/haryana-gurugram-corona...,0,1255800872266878976,NaN,NaN,NaN,und,NaN,False,0,NaN,NaN,"<a href=""http://publicize.wp.com/"" rel=""nofoll...",Haryana Gurugram Coronavirus Outbreak Live | H...,https://twitter.com/Ankit68419889/status/12558...,Sat Sep 08 09:12:38 +0000 2018,Ankit68419889,False,NaN,2,119,104,0,NaN,Jankari News,Ankit68419889,10176,NaN,https://www.jankarinews.com/,False,0
4,NaN,Fri May 01 06:56:33 +0000 2020,NaN,NaN,https://www.thisdaylive.com/index.php/2020/05/...,2,1256115272114675712,NaN,NaN,NaN,en,NaN,False,0,NaN,NaN,"<a href=""http://twitter.com/download/android"" ...",Coronavirus Outbreak: Round-the-clock Updates ...,https://twitter.com/THISDAYLIVE/status/1256115...,Mon Feb 08 11:57:51 +0000 2010,THISDAYLIVE,False,African Views on Global News,152,1415768,218,1144,Nigeria,THISDAY LIVE,THISDAYLIVE,118195,NaN,http://www.thisdaylive.com,False,0


# Description

- Favorite Count: Number of likes the tweet got 
- ID: Twitter id
- in_reply_to_status_id: which ID (status) this individual was replying to 

In [ ]:
df.describe()

,favorite_count,id,in_reply_to_status_id,in_reply_to_user_id,retweet_count,retweet_id,retweet_screen_name,user_favourites_count,user_followers_count,user_friends_count,user_listed_count,user_statuses_count,user_time_zone,label
count,2.331060e+05,2.331060e+05,1.122550e+05,1.143400e+05,233106.000000,0.0,0.0,2.331060e+05,2.331060e+05,2.331060e+05,233106.000000,2.331060e+05,0.0,233106.000000
mean,2.097276e+02,1.257931e+18,1.260020e+18,3.022486e+17,47.778641,NaN,NaN,2.448698e+04,9.136193e+04,2.130790e+03,378.921975,5.248895e+04,NaN,0.052058
std,6.586738e+03,1.445762e+16,1.508146e+16,4.746107e+17,1544.046442,NaN,NaN,5.702788e+04,1.674794e+06,1.010409e+04,4817.951184,3.738719e+05,NaN,0.222144
min,0.000000e+00,1.039642e+18,4.379533e+17,1.200000e+01,0.000000,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,NaN,0.000000
25%,0.000000e+00,1.248288e+18,1.252351e+18,6.843895e+07,0.000000,NaN,NaN,4.730000e+02,1.450000e+02,1.990000e+02,1.000000,2.887000e+03,NaN,0.000000
50%,0.000000e+00,1.256226e+18,1.256313e+18,5.863321e+08,0.000000,NaN,NaN,4.343000e+03,7.370000e+02,6.590000e+02,7.000000,1.129600e+04,NaN,0.000000
75%,2.000000e+00,1.271571e+18,1.275636e+18,8.069069e+17,0.000000,NaN,NaN,2.204800e+04,3.207000e+03,1.827000e+03,51.000000,3.718600e+04,NaN,0.000000
max,1.038569e+06,1.322627e+18,1.322548e+18,1.288902e+18,336316.000000,NaN,NaN,2.212702e+06,1.296330e+08,1.420066e+06,222979.000000,4.534897e+07,NaN,1.000000


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233106 entries, 0 to 233105
Data columns (total 35 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   coordinates                 258 non-null     object 
 1   created_at                  233106 non-null  object 
 2   hashtags                    42415 non-null   object 
 3   media                       34644 non-null   object 
 4   urls                        109285 non-null  object 
 5   favorite_count              233106 non-null  int64  
 6   id                          233106 non-null  int64  
 7   in_reply_to_screen_name     114340 non-null  object 
 8   in_reply_to_status_id       112255 non-null  float64
 9   in_reply_to_user_id         114340 non-null  float64
 10  lang                        233106 non-null  object 
 11  place                       4974 non-null    object 
 12  possibly_sensitive          124413 non-null  object 
 13  retweet_count 

# Trad ML Preprocessing

This will be the preprocessing for traditional supervised learning! Will try to extract as much info as possible from the 30+ columns. 

In [ ]:
# create a new column called user_active to show the time between when the user account was created and when the tweet was sent

# first make the created_at column a datetime object
df.created_at = pd.to_datetime(df.created_at)
df.user_created_at = pd.to_datetime(df.user_created_at)
df["user_active"] = (df.created_at - df.user_created_at).astype('timedelta64[D]')

# remove the user_created_at columns
df = df.drop(["created_at","user_created_at"],axis = 1)

In [ ]:
# filter so the language is english (lang = en)
lang_bool = df.lang == "en"
df = df.loc[lang_bool,:]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 221461 entries, 0 to 233105
Data columns (total 34 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   coordinates                 252 non-null     object 
 1   hashtags                    40892 non-null   object 
 2   media                       33124 non-null   object 
 3   urls                        104470 non-null  object 
 4   favorite_count              221461 non-null  int64  
 5   id                          221461 non-null  int64  
 6   in_reply_to_screen_name     107089 non-null  object 
 7   in_reply_to_status_id       105071 non-null  float64
 8   in_reply_to_user_id         107089 non-null  float64
 9   lang                        221461 non-null  object 
 10  place                       4724 non-null    object 
 11  possibly_sensitive          118759 non-null  object 
 12  retweet_count               221461 non-null  int64  
 13  retweet_id    

In [ ]:
df.loc[186469,:]

coordinates                                                                 NaN
hashtags                                                                    NaN
media                                                                       NaN
urls                          https://twitter.com/PascaleRobinson/status/127...
favorite_count                                                                1
id                                                          1271686118556172288
in_reply_to_screen_name                                                 Dr_Rave
in_reply_to_status_id                                               1.27134e+18
in_reply_to_user_id                                                 2.46414e+08
lang                                                                         en
place                                                                       NaN
possibly_sensitive                                                        False
retweet_count                           

In [ ]:
to_use = ["hashtags","favorite_count", "retweet_count","user_favourites_count", 
          "user_followers_count", "user_friends_count", "user_statuses_count", 
          "user_verified","place","user_listed_count","user_active","label"]
data = df.loc[:,to_use]
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 221461 entries, 0 to 233105
Data columns (total 12 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   hashtags               40892 non-null   object 
 1   favorite_count         221461 non-null  int64  
 2   retweet_count          221461 non-null  int64  
 3   user_favourites_count  221461 non-null  int64  
 4   user_followers_count   221461 non-null  int64  
 5   user_friends_count     221461 non-null  int64  
 6   user_statuses_count    221461 non-null  int64  
 7   user_verified          221461 non-null  bool   
 8   place                  4724 non-null    object 
 9   user_listed_count      221461 non-null  int64  
 10  user_active            221461 non-null  float64
 11  label                  221461 non-null  int64  
dtypes: bool(1), float64(1), int64(8), object(2)
memory usage: 30.5+ MB


In [ ]:
# working with hashtags

fake_news_hashtag = ["MaskUpMichigan","Wapo coronavirus Chinese","WuhanCoronavirus", 
                     "ChinaLiedPeopleDied","NewYorkTough JerseyStrong","COVID19 TrumpRallyTulsa",
                     "Wuhan","Trump","TrumpOwnsEveryDeath","BillGates","Fauci","BillGatesJr FireFauci IndictFauci"]

hashtag_bool = data.hashtags.isin(fake_news_hashtag)
data.loc[hashtag_bool==True, "hashtags"] = 1
data.loc[hashtag_bool == False, "hashtags"] = 0


In [ ]:
# working with place 
na_bool = data.place.isna()
data.loc[na_bool == True, "place"] = 0 
data.loc[na_bool == False, "place"] = 1

In [ ]:
data["hashtags"] = data.hashtags.astype("str").astype("int")
data["place"] = data.place.astype("str").astype("int")

In [ ]:
# make dummy variables 
data.user_verified = pd.Categorical(data.user_verified, categories = [False, True])
data.hashtags = pd.Categorical(data.hashtags, categories = [0,1])
data.place = pd.Categorical(data.place, categories = [0,1])
data = pd.get_dummies(data, drop_first = True)

In [ ]:
data.describe()

,favorite_count,retweet_count,user_favourites_count,user_followers_count,user_friends_count,user_statuses_count,user_listed_count,user_active,label,hashtags_1,user_verified_True,place_1
count,2.214610e+05,221461.000000,2.214610e+05,2.214610e+05,2.214610e+05,2.214610e+05,221461.000000,221461.000000,221461.000000,221461.000000,221461.000000,221461.000000
mean,2.178703e+02,49.440651,2.435581e+04,9.315908e+04,2.163814e+03,4.992278e+04,391.720493,2350.849043,0.047751,0.000239,0.080719,0.021331
std,6.750118e+03,1580.903186,5.705149e+04,1.703534e+06,1.032495e+04,1.595496e+05,4930.696824,1409.120688,0.213240,0.015468,0.272403,0.144486
min,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000e+00,0.000000,4.520000e+02,1.450000e+02,2.000000e+02,2.883000e+03,1.000000,1077.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000e+00,0.000000,4.227000e+03,7.460000e+02,6.660000e+02,1.122600e+04,7.000000,2582.000000,0.000000,0.000000,0.000000,0.000000
75%,2.000000e+00,0.000000,2.172700e+04,3.255000e+03,1.843000e+03,3.702600e+04,53.000000,3639.000000,0.000000,0.000000,0.000000,0.000000
max,1.038569e+06,336316.000000,2.212702e+06,1.296330e+08,1.420066e+06,6.664798e+06,222979.000000,5196.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
trad_ml = data.copy()
trad_ml.to_csv("/content/drive/MyDrive/UBDC_2021/Final/trad_ml.csv")

# Text Preprocessing

Now we will explore preprocessing but for the actual text & column labels only. This will be for the NLP methods. 

*Referring to Mashi's code here*

In [ ]:
data2 = df.loc[:,["text","label"]]
data2.head()

,text,label
0,Coronavirus Outbreak LIVE Updates: Odisha conf...,0
1,"For today, May 1, here’s how to follow the lat...",0
4,Coronavirus Outbreak: Round-the-clock Updates ...,0
5,WATCH LIVE: Gov. Phil Murphy will speak at 1 p...,0
8,To all my friends in the people's republic of ...,0


In [ ]:
# clean the text 
cleaned_words = []
num_tweets = data2.shape[0]
print(num_tweets)

for i in range(0, num_tweets):
   tweet = re.sub(r"http\S+", "", data2.iloc[i,0]) # remove all URLs
   tweet = re.sub('[^a-zA-z]',' ',tweet) # remove punctuation
   tweet = re.sub(r'@\S+|https?://\S+','', tweet) # remove @ sign
   tweet = tweet.lower() # make all letters lower case
   #tweet = tweet.split() # make a list of the words

   # now will stem words 
   #ps = PorterStemmer()
   #all_stopwords = stopwords.words("english")
   #all_stopwords.remove("not") # make sure we don't remove the word "not" since that changes the meaning of the sentence
   #tweet = [ps.stem(t) for t in tweet if not t in set(all_stopwords)]
   #tweet = " ".join(tweet) # join the words back together
   cleaned_words.append(tweet) # app cleaned tweets to the original list

   #ps = PorterStemmer()
   #tweet = [ps.stem(t) for t in tweet]
   #tweet = " ".join(tweet) # join the words back together
   #cleaned_words.append(tweet) # app cleaned tweets to the original list

221461


In [ ]:
print(len(cleaned_words))

221461


In [ ]:
# make the dataframe again 
final_df = pd.DataFrame(data = {"text":cleaned_words, "label":data2.label})
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 221461 entries, 0 to 233105
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    221461 non-null  object
 1   label   221461 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 15.1+ MB


In [ ]:
final_df.groupby("label").label.count()

label
0    210886
1     10575
Name: label, dtype: int64

In [ ]:
final_df.head()

,text,label
0,coronavirus outbreak live updates odisha conf...,0
1,for today may here s how to follow the lat...,0
4,coronavirus outbreak round the clock updates ...,0
5,watch live gov phil murphy will speak at p...,0
8,to all my friends in the people s republic of ...,0


In [ ]:
final_df.loc[58464,:] # the text was all stop words!

text     what can you do 
label                   0
Name: 58464, dtype: object

In [ ]:
# we've got some empty strings that need removing
final_df.text.replace("", np.nan, inplace= True)
final_df.dropna(how = "any",inplace = True)
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 221461 entries, 0 to 233105
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    221461 non-null  object
 1   label   221461 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 5.1+ MB


In [ ]:
# export
final_df.to_csv("/content/drive/MyDrive/UBDC_2021/Final/text_preprocessed.csv")

In [ ]:
data2 = df.loc[:,["text","label"]]
data2.head()

# clean the text 
cleaned_words = []
num_tweets = data2.shape[0]
print(num_tweets)

for i in range(0, num_tweets):
   tweet = re.sub(r"http\S+", "", data2.iloc[i,0]) # remove all URLs
   tweet = re.sub('[^a-zA-z]',' ',tweet) # remove punctuation
   tweet = re.sub(r'@\S+|https?://\S+','', tweet) # remove @ sign
   tweet = tweet.lower() # make all letters lower case
   tweet = tweet.split() # make a list of the words

   # now will stem words 
   ps = PorterStemmer()
   all_stopwords = stopwords.words("english")
   all_stopwords.remove("not") # make sure we don't remove the word "not" since that changes the meaning of the sentence
   tweet = [t for t in tweet if not t in set(all_stopwords)] # don't stem
   tweet = " ".join(tweet) # join the words back together
   cleaned_words.append(tweet) # app cleaned tweets to the original list

# make the dataframe again 
final_df = pd.DataFrame(data = {"text":cleaned_words, "label":data2.label})
final_df.info()

# we've got some empty strings that need removing
final_df.text.replace("", np.nan, inplace= True)
final_df.dropna(how = "any",inplace = True)
final_df.info()

final_df.to_csv("/content/drive/MyDrive/UBDC_2021/Final/text_preprocessed_rnn.csv")

221461
<class 'pandas.core.frame.DataFrame'>
Int64Index: 221461 entries, 0 to 233105
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    221461 non-null  object
 1   label   221461 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 15.1+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 221444 entries, 0 to 233105
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    221444 non-null  object
 1   label   221444 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 5.1+ MB
